# First Attempt (3 - 9 July 2022)

I spent a few days playing sodoku and trying to find patterns in the way I approached solving the problem. The first thought I had was evaluating an empty cell by looking at its row and column and box one by one and seeing if I could fill in a correct answer that way. My thought was that in order to ensure that I make the computer only input correct answers only, I would have the program go to an empty cell, look at the row or column or box it was in, then see if there were 8 out of 9 that were already filled in. That way, all the program would have to do would be to fill in the remaining number. That logic would loop over and over until all the cells were filled in and the puzzle would be solved.

I tried to replicate that logic by hand on an empty sodoku, but realized that there are very few sodoku puzzles that had initial conditions that favored that approach. You'd have to be lucky to get a puzzle whose initial condition had a row/column/box that had 8 out of 9 already filled in, and those were the easiest of the easy sodokus. If you were not lucky, the program would just continue to infinitely loop trying to find a single solution, but never solving the puzzle. 

# First Attempt: Part II (10 - 16 July 2022)

After playing some more sodokus by hand and messing around with solving techniques on a whiteboard, I found a better approach. What if for every empty sodoku cell, the program evaluated what number that cell COULD be, based on the rest of the cells in its same row/column/box? That row, column, and box would have their own list of possible values. That way, you could compare those lists of possible values to one another. If there exists a SINGLE UNIQUE number that all three share, that has to be the solution for that cell. Effectively, the program compares the intersection of sets to find a solution. The program would loop through every empty cell and repeat that logic until all cells are filled in. 

The code below reflects this approach.

### Import Dependencies

In [1]:
import numpy as np

In [2]:
#This list will be used to help identify if a row/column/box already has these numbers
num = [1,2,3,4,5,6,7,8,9]

### Row & Column Lists

In order to start finding a solution for an empty cell, we need to evaluate its neighbors in its row, its column, and its box. These functions below are necessary to turn a row, column, or box into a single SET of numbers. 

*A set is necessary instead of a list because zero shows up multiple times in a row/column/box and all we want is for that zero to show up once.* 

In [25]:
def row_list(b,r): return set([i for i in b[r,:]])
#Inputs: Takes a 9x9 2d Numpy array, a specified row
#Returns: Set of numbers in that row (from 0 to 9)

In [26]:
def column_list(b,c): return set([i for i in b[:,c]])
#Inputs: Takes a 9x9 2d Numpy array, a specified column
#Returns: Set of numbers in that column (from 0 to 9)

### Box Logic

The logic for boxes is more involved because boxes span over specific rows and columns. The best approach that I came to was by associating every cell with a specific box using numpy's dstack function, which effectively works like a zip function. Then using a function that evaluates the whole puzzle and only returns a value to a list if that value is within that box.

In [3]:
boxes = np.array([[1,1,1,2,2,2,3,3,3],
                  [1,1,1,2,2,2,3,3,3],
                  [1,1,1,2,2,2,3,3,3],
                  [4,4,4,5,5,5,6,6,6],
                  [4,4,4,5,5,5,6,6,6],
                  [4,4,4,5,5,5,6,6,6],
                  [7,7,7,8,8,8,9,9,9],
                  [7,7,7,8,8,8,9,9,9],
                  [7,7,7,8,8,8,9,9,9]])

def box_number(b,r,c): 
    """Inputs: 9x9 2d np array, row index of target cell, column index of cell
       Returns: the box number of that cell"""
    stacked = np.dstack((b,boxes))
    return stacked[r,c,1]

def box_list(b,n):
    """Inputs: 9x9 2d np array, number of the box you want to evaluate
       Returns: A set of numbers that are in that target box"""
    stacked = np.dstack((b,boxes))
    return set([j[0] for i in stacked for j in i if j[1] == n])

### Determining possible values for an empty cell

Now that the list functions are set up, we can set up functions that return possible values for an empty cell, given the values in its respective row/column/box. Each function below evaluates what's in the row/column/box, then removes that value from the list of numbers from 0 to 9. The remaining numbers are what that empty cell *could* be.

In [27]:
def p_row(b,r):
    """Inputs: 9x9 2d np array, row index of target cell
       Returns: set of numbers that the empty cell could be from 1 to 9"""
    row = set(num)
    array_row = row_list(b,r)
    for i in array_row:
        if i in row:
            row.remove(i)
    return row

In [28]:
def p_column(b,c):
    """Inputs: 9x9 2d np array, column index of target cell
       Returns: set of numbers that the empty cell could be from 1 to 9"""
    column = set(num)
    array_column = column_list(b,c)
    for i in array_column:
        if i in column:
            column.remove(i)
    return column

In [29]:
def p_box(b,x):
    """Inputs: 9x9 2d np array, box number of target cell
       Returns: set of numbers that the empty cell could be from 1 to 9"""
    box = set(num)
    array_box = box_list(b,x)
    for i in array_box:
        if i in box:
            box.remove(i)
    return box

### Intersection of Sets

This next function takes the sets of the possible values of the target cell's row, column, and box, and uses its intersection as the cell's total possible solution. This works because while there may be numbers the empty cell could be based on its respective row, that number may already be in its respective column or box, therefore that number is not a valid answer. 

A perfect solution exists if and only if the output of the intersection is a single element.

In [30]:
def solution(b,r,c):
    """Inputs: 9x9 2d np array, row index of empty cell, column index of empty cell
       Returns: set of the correct solution for that empty cell, otherwise False"""
    x = p_row(b,r)
    y = p_column(b,c)
    z = p_box(b,box_number(b,r,c))
    could_be = x & y & z
    if len(x & y & z) <= 1 : 
        return could_be
    else: return False 

### Putting it all together 

Using the above function, we can now loop over the whole sodoku puzzle cell by cell and determine solutions and update the sodoku. 

In this case, a while loop in conjunction with a for loop is appropriate. I initially set my while conditions as "as long as there exists empty cells (cell's value as zero) in the sodoku puzzle, keep looping". I found out that this is an incorrect approach because the harder the puzzles get, the solver logic stops finding solutions and the loop keeps runing indefinitely. The better answer was to include a counter and when the loop finds a solution to an empty cell, add to that counter and subtract from it on the next iteration. As long as the solver continues to find solutions, the loop will continue, but if it fails to find one, it will ultimately break and let you know that it failed to solve the puzzle. 

In [32]:
def solver(sodoku_list, answer):
    """Inputs: unsolved sodoku puzzle in a 9x9 2d np array, its correct solution in a 9x9 2d np array
       Returns: A solved sodoku puzzle in array format along with True if successful, or its best attempt along with False, indicating the solver failed"""
    clone = sodoku_list
    count = 1
    while count > 0: 
        count -= 1
        for i in range(9):
            for j in range(9):
                if clone[i,j] != 0:
                    continue
                else:
                    solved = solution(clone,i,j)
                    if solved != False:
                        clone[i,j] = int(list(solved)[0])
                        count += 1 
                        
    return clone, np.array_equiv(clone,answer)

### Testing

With the solver set up, we can now test how it performs on actual sodoku puzzles. I pulled these from sodoku.com and inputted them by hand into a numpy array along with their correct solutions so we can compare if the solver was accurate or not. 

In [13]:
#Easy
test_1 = np.array([[7,4,0,0,3,0,0,1,0],
                   [0,1,9,0,6,8,5,0,2],
                   [0,0,0,0,0,4,3,0,0],
                   [0,5,6,3,7,0,0,0,1],
                   [0,0,1,8,0,0,0,9,5],
                   [0,9,0,0,2,0,6,0,0],
                   [1,0,3,4,0,7,2,0,0],
                   [5,0,0,2,0,0,0,0,8],
                   [0,8,0,0,0,1,4,7,0]])

test_1_answer = np.array([[7,4,8,5,3,2,9,1,6],
                          [3,1,9,7,6,8,5,4,2],
                          [6,2,5,9,1,4,3,8,7],
                          [4,5,6,3,7,9,8,2,1],
                          [2,3,1,8,4,6,7,9,5],
                          [8,9,7,1,2,5,6,3,4],
                          [1,6,3,4,8,7,2,5,9],
                          [5,7,4,2,9,3,1,6,8],
                          [9,8,2,6,5,1,4,7,3]])




In [33]:
solver(test_1,test_1_answer)

(array([[7, 4, 8, 5, 3, 2, 9, 1, 6],
        [3, 1, 9, 7, 6, 8, 5, 4, 2],
        [6, 2, 5, 9, 1, 4, 3, 8, 7],
        [4, 5, 6, 3, 7, 9, 8, 2, 1],
        [2, 3, 1, 8, 4, 6, 7, 9, 5],
        [8, 9, 7, 1, 2, 5, 6, 3, 4],
        [1, 6, 3, 4, 8, 7, 2, 5, 9],
        [5, 7, 4, 2, 9, 3, 1, 6, 8],
        [9, 8, 2, 6, 5, 1, 4, 7, 3]]),
 True)

For an easy sodoku, it works pretty well and returns an answer instantaneously.

In [35]:
#Trying another easy sodoku just in case the first one was a fluke.

#Easy
test_2 = np.array([[2,0,0,0,9,0,0,4,5],
                   [0,9,0,0,5,1,8,0,2],
                   [7,5,0,4,0,0,0,0,0],
                   [0,0,0,0,0,0,3,0,8],
                   [1,0,0,3,6,7,4,0,0],
                   [0,7,4,8,0,0,0,0,1],
                   [5,3,1,2,0,6,0,0,4],
                   [8,2,0,9,4,5,6,0,3],
                   [0,4,0,0,0,8,0,0,0]])

test_2_answer = np.array([[2,1,8,6,9,3,7,4,5],
                          [4,9,6,7,5,1,8,3,2],
                          [7,5,3,4,8,2,1,9,6],
                          [9,6,2,5,1,4,3,7,8],
                          [1,8,5,3,6,7,4,2,9],
                          [3,7,4,8,2,9,5,6,1],
                          [5,3,1,2,7,6,9,8,4],
                          [8,2,7,9,4,5,6,1,3],
                          [6,4,9,1,3,8,2,5,7]])

In [36]:
solver(test_2,test_2_answer)

(array([[2, 1, 8, 6, 9, 3, 7, 4, 5],
        [4, 9, 6, 7, 5, 1, 8, 3, 2],
        [7, 5, 3, 4, 8, 2, 1, 9, 6],
        [9, 6, 2, 5, 1, 4, 3, 7, 8],
        [1, 8, 5, 3, 6, 7, 4, 2, 9],
        [3, 7, 4, 8, 2, 9, 5, 6, 1],
        [5, 3, 1, 2, 7, 6, 9, 8, 4],
        [8, 2, 7, 9, 4, 5, 6, 1, 3],
        [6, 4, 9, 1, 3, 8, 2, 5, 7]]),
 True)

Works as expected again on another easy problem. Now to try more difficult tests.

In [15]:
#Medium
test_3 = np.array([[5,6,8,7,1,4,0,0,0],
                   [0,0,0,6,0,0,0,5,8],
                   [0,0,0,5,0,0,0,0,0],
                   [9,0,0,0,0,0,5,0,0],
                   [0,3,5,2,0,7,8,0,0],
                   [7,0,1,0,5,0,0,6,0],
                   [0,0,0,0,6,2,7,0,0],
                   [2,0,9,4,7,0,0,0,0],
                   [4,7,0,0,0,0,2,1,0]])

test_3_answer = np.array([[5,6,8,7,1,4,9,3,2],
                          [1,9,7,6,2,3,4,5,8],
                          [3,4,2,5,9,9,1,7,6],
                          [9,8,4,1,3,6,5,2,7],
                          [6,3,5,2,4,7,8,9,1],
                          [7,2,1,8,5,9,3,6,4],
                          [8,1,3,9,6,2,7,4,5],
                          [2,5,9,4,7,1,6,8,3],
                          [4,7,6,3,8,5,2,1,9]])

In [37]:
solver(test_3,test_3_answer)

(array([[5, 6, 8, 7, 1, 4, 0, 0, 0],
        [0, 0, 0, 6, 0, 0, 0, 5, 8],
        [0, 0, 0, 5, 0, 0, 0, 0, 0],
        [9, 0, 0, 0, 0, 0, 5, 0, 0],
        [6, 3, 5, 2, 0, 7, 8, 0, 0],
        [7, 0, 1, 0, 5, 0, 0, 6, 0],
        [0, 0, 3, 0, 6, 2, 7, 0, 0],
        [2, 0, 9, 4, 7, 0, 0, 0, 0],
        [4, 7, 6, 0, 0, 0, 2, 1, 0]]),
 False)

As we can see, the solver failed. The solver's solution only found a few solutions to empty cells but hit a wall when it couldn't find any more, therefore breaking the loop. 

Let's continue on and try a hard one just to see how it performs:

In [16]:
#Hard 
test_4 = np.array([[7,0,0,0,9,0,0,8,0],
                   [0,3,0,0,6,0,0,0,4],
                   [0,0,0,0,0,0,0,0,0],
                   [2,0,0,0,0,0,0,0,5],
                   [8,0,0,0,2,6,4,0,0],
                   [6,0,0,0,7,0,0,0,3],
                   [0,1,0,0,4,0,0,0,9],
                   [0,0,0,2,0,5,0,0,0],
                   [0,7,0,9,0,0,5,0,0]])

test_4_answer = np.array([[7,2,5,3,9,4,6,8,1],
                          [1,3,9,8,6,2,7,5,4],
                          [4,8,6,7,5,1,3,9,2],
                          [2,4,7,1,8,3,9,6,5],
                          [8,9,3,5,2,6,4,1,7],
                          [6,5,1,4,7,9,8,2,3],
                          [5,1,8,6,4,7,2,3,9],
                          [9,6,4,2,3,5,1,7,8],
                          [3,7,2,9,1,8,5,4,6]])

In [38]:
solver(test_4,test_4_answer)

(array([[7, 0, 0, 0, 9, 0, 0, 8, 0],
        [0, 3, 0, 0, 6, 0, 0, 0, 4],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0, 0, 5],
        [8, 0, 0, 0, 2, 6, 4, 0, 0],
        [6, 0, 0, 0, 7, 0, 0, 0, 3],
        [0, 1, 0, 0, 4, 0, 0, 0, 9],
        [0, 0, 0, 2, 0, 5, 0, 0, 0],
        [0, 7, 0, 9, 0, 0, 5, 0, 0]]),
 False)

Once again, the solver failed to find a full solution. Actually, the initial conditions were so empty that the loop didn't even find a single cell's solution. 

### Iteration 1 Conclusion 

As tests 3 and 4 show, the solver logic needs some improvement. It can solve easy sodoku's no problem, but fails to solve the medium and harder ones. 

For the next iteration, I plan to add some more logic to the solver. Specifically, I'll try having the solver evaluate the empty cell's *adjacent* rows and columns instead of just its own. 

Thanks again for checking out the first iteration of my second project.

Feel free to send me some feedback or pointers to my email at cameron.fld@gmail.com